# Page Rank

## Import Package

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tqdm.notebook import tqdm
from copy import deepcopy
import pickle

## EdgeList class

In [36]:
def  Normalize2P(P, i, n, norm_1): 
  if i == None:
    for u in P:
      P[u] += (1 - norm_1) / n
  else:
      P[i] += (1 - norm_1)
  return P

In [42]:
class EdgeList:
  '''
  Class to model a graph using edgelist structure.
  This model used for the PageRank algorithm.
  '''
  def read_file_name(self, file_name):
    '''
      Read the names of nodes .
      Input:
        - file_name: file to read the node name pair.
      Add these attributes:
        - names: dict for each node it's name.
        - nodes: list of nodes in the graph.
        - n: number of nodes.
    '''
    self.names = {}
    self.nodes = []
    with(open(file_name, 'r')) as f:
      line = f.readline()
      while line:
        line = line.split()
        if len(line) != 0 and line[0] != "#":
          u = int(line[0])
          self.nodes.append(u)
          self.names[u] = " ".join(line[1:])
        line = f.readline()
    f.close()
    self.n = len(self.nodes)

  def read_file_edglist(self, file_name):
    '''
      Read a graph from file and store it as edgelist
      Input:
        - file_name: file to read the graph from.
      Add these attributes:
        - n: number of nodes.
        - m: number of edges.
        - edgelist: a dict with keys the nodes and values
                    list of neighbors.
        - deg_out: dict with keys the nodes
                  and value the out degree.
        - deg_in: dict with keys the nodes
                  and value the in degree.
    '''
    self.m = 0
    self.edgelist ={}
    with(open(file_name, 'r')) as f:
      line = f.readline()
      while line:
        line = line.split()
        if len(line) == 2:
          u = int(line[0])
          v = int(line[1])
          self.m += 1
          if not(u in self.edgelist.keys()):
            self.edgelist[u] = []
          self.edgelist[u].append(v)            
        line = f.readline()
    f.close()
    self.deg_out = {}
    self.deg_in = {}
    for u in self.nodes:
      self.deg_in[u] = 0
      if u in self.edgelist.keys():
        self.deg_out[u] = len(self.edgelist[u])
      else:
        self.deg_out[u] = 0
    for u, neighbors in self.edgelist.items():
      for v in neighbors:
        self.deg_in[v] +=1

  def __init__(self, file_names, file_graph):
    self.read_file_name(file_names)
    self.read_file_edglist(file_graph)
    self.reverse_edgelist()
    self.get_end_nodes()

  def edge_exists(self, u, v):
    '''
     Return if there's an edge from u to v.
     Input:
      - edgelist: a graph stored as edgelist
      - u: starting node.
      - v: ending node.
    Output:
      - Boolean if u lead to v.
    '''
    return v in self.edgelist[u]
  
  def reverse_edgelist(self):
    '''
    This function compute the pagerank algorithm based on the power iteration method.
      Inputs:
          - n: number of nodes.
          - m: number of edges.
          - edgelist: a dict with keys the nodes and values
                      list of neighbors.
          - nodes: list of nodes.
        Add this attribute:
        - reverse_edgelist: a dict with keys the nodes and values
                      list of nodes that lead to the node.
    '''
    self.reverse_edgelist = {}
    for u in self.nodes:
      if not u in self.reverse_edgelist.keys():
        self.reverse_edgelist[u] = []
      if not u in self.edgelist.keys():
        continue
      for v in self.edgelist[u]:
        if not (v in self.reverse_edgelist.keys()):
          self.reverse_edgelist[v] = []
        self.reverse_edgelist[v].append(u)

  def get_end_nodes(self):
    '''
      This function compute the pagerank algorithm based on the power iteration method.
      Inputs:
          - edgelist: a dict with keys the nodes and values
                      list of neighbors.
          - nodes: list of nodes.
          - deg_out: dict contains the out degree of every node.
        Add this attribute:
        - end_nodes: a list of nodes that have an out degree equal to 0.
    '''
    self.end_nodes = []
    for u in self.nodes:
      if self.deg_out[u] == 0:
        self.end_nodes.append(u)
  def PageRank(self, alpha= 0.1, t= 100, epsilion= 1e-3, P0= None):
    '''
     This method compute the pagerank algorithm based on the power iteration 
     method.
     Inputs:
        - alpha: damping factor, between 0 and 1.
        - t: number of iterations to be done.
        - epsilion: the limit of convergence of the graph.
        - P0: node for the rooted pagerank.
              If not initialized the regular pagerank will be used. 
     Output:
      - P:  dict of Pageranks with size n (number of node).
      - history: dict containing the different values for each iterations.
      - total_time: the time needed to finsih all iterations/converge.
    '''
    P = {u:1/self.n for u in self.nodes}
    Pt = {u:0 for u in self.nodes}
    start_time = time.time()
    history = {}
    for i in range(t):
        iter_time = time.time()
        norm1 = np.sum([abs(P[u]) for u in self.end_nodes]) / self.n
        end_inter = np.sum([P[u] for u in self.end_nodes]) / self.n
        for v in tqdm(self.nodes):
            inter = end_inter + np.sum([P[u]/self.deg_out[u] for u in self.reverse_edgelist[v]])
            if P0 == None:
                Pt[v] = (1 - alpha) * inter + alpha /self.n
            elif v in P0.keys():
                Pt[v] = (1 - alpha) * inter + alpha * P0[v]
            else:
                Pt[v] = (1 - alpha) * inter
            norm1 += abs(Pt[v]) # compute the norm 1
        Pt = Normalize2P(Pt, P0, self.n, norm1)
        diff = np.sum([abs(Pt[u] - P[u]) for u in self.nodes])
        P = deepcopy(Pt)
        history[i] = {}
        history[i]["norm"] = norm1
        history[i]["diff"] = diff
        history[i]["time"] = time.time() - iter_time
        if diff < epsilion:
            break
    total_time = time.time() - start_time
    return P, history, total_time

In [43]:
path = "../../data"
file_name_edge = path + "/alr21--dirLinks--enwiki-20071018.txt"
file_name_names = path + "/alr21--pageNum2Name--enwiki-20071018.txt"

In [39]:
G = EdgeList(file_name_names, file_name_edge)

In [40]:
alpha = 0.15 
t = 20
eps = 1e-3
P, history, total_time = G.PageRank(alpha= alpha, t= t, epsilion= eps)

TypeError: 'int' object is not iterable

In [ ]:
total_time / 60

We can see that after 12 iterations the $|| P_{t+1} - P_{t}||_1 
< \epsilon$ with $\epsilon = 0.001$

In [ ]:
history = pd.DataFrame.from_dict(history)
history = history.transpose()
history = history.reset_index()
history = history.rename(columns= {"index": "iteration"})
history.head()

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.lineplot(data= history, y="diff", x="iteration", label= "$|| P_{t+1} - P_{t} ||_1$")
plt.plot([eps for i in range(history.shape[0])], label= "$\epsilon$= "+ str(eps))
plt.title("Convergence of the PageRank algorithm")
plt.xlabel("iteration")
plt.ylabel(" $|| P_{t+1} - P_{t} ||_1$")
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.lineplot(data= history, y="time", x="iteration")
plt.title("PageRank algorithm: time per algorithm")
plt.xlabel("iteration")
plt.ylabel("time (seconds)")
plt.legend()
plt.show()

#### Top 5 pages

In [ ]:
Values = sorted(list(P.values()), reverse= True)
Val5 = Values[:5]
Top5 = []
Top5id = []
values5 = []
for val in Val5:
  for u, valeur in P.items():
    if valeur == val:
      Top5.append(G.names[u])
      Top5id.append(u)
      values5.append(valeur)
      break
TOP = pd.DataFrame([Top5, Top5id, values5])
print("Top5: ", Top5)
print("Top5id: ", Top5id)
print("Values5: ", values5)
TOP = TOP.transpose().rename(columns={0: "name",
                                      1: "ID",
                                      2: "PageRank"})
TOP

#### low 5 pages

In [ ]:
Val5 = Values[-1]
Low5 = []
Low5id = []
for u, val in P.items():
  if val == Val5:
    Low5.append(G.names[u])
    Low5id.append(u)
LOW = pd.DataFrame([Low5[0:5], Low5id[0:5], [Val5 for i in range(5)]])
print("Low5: ", Low5[0:5])
print("Low5id: ", Low5id[0:5])
LOW = LOW.transpose().rename(columns={0: "name",
                                      1: "ID",
                                      2: "PageRank"})
LOW

## Exercice 2

In [ ]:
P1, history1, total_time1 = G.PageRank(alpha= 0.1, t= 20)

In [ ]:
P2, history2, total_time2 = G.PageRank(alpha= 0.2, t= 20)

In [ ]:
P3, history3, total_time3 = G.PageRank(alpha= 0.5, t= 20)

In [ ]:
P4, history4, total_time4 = G.PageRank(alpha= 0.9, t= 20)

In [ ]:
data = pd.DataFrame.from_dict(G.deg_in, orient= "index", columns= ["in-degree"])
data = data.join(pd.DataFrame.from_dict(G.deg_out, orient= "index", columns= ["out-degree"]))
data = data.join(pd.DataFrame.from_dict(G.names, orient= "index", columns= ["name"]))
data = data.join(pd.DataFrame.from_dict(P, orient= "index", columns= ["PageRank"]))
data = data.join(pd.DataFrame.from_dict(P1, orient= "index", columns= ["PageRank0.1"]))
data = data.join(pd.DataFrame.from_dict(P2, orient= "index", columns= ["PageRank0.2"]))
data = data.join(pd.DataFrame.from_dict(P3, orient= "index", columns= ["PageRank0.5"]))
data = data.join(pd.DataFrame.from_dict(P4, orient= "index", columns= ["PageRank0.9"]))
data = data.reset_index()
data = data.rename(columns= {"index": "ID"})
data

In [ ]:
save_data = True

In [ ]:
if save_data == True:
    file_data = path + "/data_sim.pkl"
    with open(path+file_data, 'wb') as f:
        pickle.dump(data, f)
    f.close()

In [ ]:
load_data = False

In [ ]:
if load_data == True:
    file_data = path + "/data_sim.pkl"
    with open(path+file_data, 'rb') as f:
        data = pickle.load(f)
    f.close()

#### PageRank VS in-degree

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.scatterplot(data= data, x="PageRank", y="in-degree")
plt.title("Scatter plot")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.scatterplot(data= data, x="PageRank", y="in-degree")
#ax.set_yscale('log')
ax.set_xscale('log')
plt.title("Scatter plot log")
plt.show()

#### PageRank VS out-degree

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.scatterplot(data= data, x="PageRank", y="out-degree")
# plt.xticks(rotation = 45)
plt.title("Scatter plot")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.scatterplot(data= data, x="PageRank", y="out-degree")
#ax.set_yscale('log')
ax.set_xscale('log')
plt.title("Scatter plot log")
plt.show()

#### PageRank0.15 VS PageRank0.1

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.scatterplot(data= data, x="PageRank", y="PageRank0.1")
# plt.xticks(rotation = 45)
plt.title("Scatter plot")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.scatterplot(data= data, x="PageRank", y="PageRank0.1")
#ax.set_yscale('log')
ax.set_xscale('log')
plt.title("Scatter plot log")
plt.show()

#### PageRank0.15 VS PageRank0.2

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.scatterplot(data= data, x="PageRank", y="PageRank0.2")
# plt.xticks(rotation = 45)
plt.title("Scatter plot")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.scatterplot(data= data, x="PageRank", y="PageRank0.2")
#ax.set_yscale('log')
ax.set_xscale('log')
plt.title("Scatter plot log")
plt.show()

#### PageRank0.15 VS PageRank0.5

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.scatterplot(data= data, x="PageRank", y="PageRank0.5")
# plt.xticks(rotation = 45)
plt.title("Scatter plot")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.scatterplot(data= data, x="PageRank", y="PageRank0.5")
ax.set_yscale('log')
ax.set_xscale('log')
plt.title("Scatter plot log")
plt.show()

#### PageRank0.15 VS PageRank0.9

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.scatterplot(data= data, x="PageRank", y="PageRank0.9")
# plt.xticks(rotation = 45)
plt.title("Scatter plot")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.scatterplot(data= data, x="PageRank", y="PageRank0.9")
ax.set_yscale('log')
ax.set_xscale('log')
plt.title("Scatter plot log")
plt.show()

### Correlation matrix

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
corrMatrix = data.drop(columns="ID").corr()
sns.heatmap(corrMatrix, annot=True)
plt.yticks(rotation = 0)
plt.show()

In [ ]:
data[data["ID"] == 3434750 ]

## Exercice 3 

In [ ]:
data[data["name"] == "Magnus Carlsen"]

In [ ]:
u = data.loc[data["name"] == "Magnus Carlsen"]["ID"].item()
P0 = {node:0 for node in G.nodes()}
P0[u] = 1
alpha = 0.15
t = 10
eps = 0

In [ ]:
PMC = G.PageRank(alpha= alpha, t= t, epsilion= eps, P0= P0)

In [ ]:
Values = sorted(list(PMC.values()), reverse= True)
Val5 = Values[:5]
Top5 = []
Top5id = []
values5 = []
for val in Val5:
  for u, valeur in PMC.items():
    if valeur == val:
      Top5.append(G.names[u])
      Top5id.append(u)
      values5.append(valeur)
      break
print("Top5: ", Top5)
print("Top5id: ", Top5id)
print("Values5: ", values5)